# v3t3p2ph2_pso_gene.ipynb

Hyperparameter search on the v3_from_t3p2_ph2 model using either Particle Swarm Optimization, a genetic algorithm, or a combinatin of both using two machines.

When running both algorithms, one machine maintains a population evolved through the PSO algorithm, whereas the second machine uses a genetic algorithm. They share their configurations through a shared file system.

In [1]:
%cd ../..
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
from datetime import datetime
from multiprocessing import Pool

/home/z/projects/draculab


In [2]:
%load_ext Cython

In [3]:
%%cython
from draculab import *

In [4]:
%cd /home/z/projects/draculab/notebook/spinal/
from v3ft3p2ph2_net import *
%cd ../..

/home/z/projects/draculab/notebook/spinal
/home/z/projects/draculab


In [5]:
# Utility methods, ranges, search configuration parameters

focus_params = True # whether to focus mutations on a specific set of parameters
max_evals = 8 # maximum number of times to evaluate a configuration
target_fitness = 0.02 # stop if fitness reaches this value

# Specify paramters and ranges
# defaults are based on incumbent 9 of smac_pars in v3_normal_smac_test.ipynb
ranges = {"A__M_lrate" : {"low": 0.1, "high": 20., "default":8.},
          "A__M_w_max_frac" : {"low": .05, "high": 1., "default":.33},
          "A__M_w_sum" : {"low": 1., "high": 40., "default":2.},
          "AL_thresh" : {"low": -.1, "high": 1., "default":.4},
          "b_e": {"low": .5, "high": 10., "default":2.},
          "C__C_antag": {"low": 0.1, "high": 3., "default":1.5},
          "C__C_p_antag": {"low": 0., "high": 1.5, "default":.25},
          "C__C_p_syne": {"low": 0., "high": 1., "default":.3},
          "C__C_syne": {"low": 0., "high": 2., "default":.5},
          "C_adapt_amp": {"low": 0., "high": 15., "default":5.},
          "C_cid" : {"low": 0.1, "high": 2., "default":.15},
          "C_sigma" : {"low": 0., "high": 1., "default":.3},
          "C_slope" : {"low": 0.5, "high": 4., "default":2.5},
          "C_tau" : {"low": 0.01, "high": .3, "default":.05},
          "C_tau_slow" : {"low": 2., "high": 40., "default":10.},
          "C_thresh" : {"low": -0.2, "high": 1.5, "default":.7},
          "CE__CI_w": {"low": 0., "high": 2.5, "default":.5},
          "CI__CE_w": {"low": -2.5, "high": 0, "default":-1.5},
          "g_e_factor": {"low": 0.5, "high": 4., "default":2.},
          "M__C_lrate" : {"low": 5., "high": 500., "default":300.},
          "M__C_w_sum": {"low": 0.5, "high": 8., "default": 2.5},
          "M__M_w": {"low": 0., "high": -3., "default":-.5},
          "M_cid": {"low": 0.05, "high": 2., "default": .3},
          "M_des_out_w_abs_sum": {"low": 0.5, "high": 4., "default": 2.},
          "SPF__SPF_w": {"low": -3., "high": 0., "default":-.5}
         }


#par_list = [name for name in ranges] # ordered list with names of the parameters
par_list = list(ranges.keys())
# parameters to focus on
main_pars = ["A__M_w_max_frac", "A__M_w_sum",  "C_adapt_amp", "C_cid",
             "C_sigma", "M_cid", "M_des_out_w_abs_sum",
             "g_e_factor", "C_slope", "C_thresh", "C_tau", "C_tau_slow"]
if focus_params:
    par_list = main_pars

def mutate(cfg, name_list=par_list):
    """ Mutate a single parameter of the given configuration. 
    
        Args:
            name_list: list with names of candidate parameters.
    """
    n = np.random.randint(len(name_list))
    par_name = name_list[n]
    l = ranges[par_name]['low']
    h = ranges[par_name]['high']
    cfg[par_name] =  l + (h-l)*np.random.random()
    
def soft_mutate(cfg, ma, name_list=par_list):
    """ Soft-mutate a single parameter of the given configuration.
    
        A 'soft mutation' keeps the mutated value close to the original value.
        The maximum amplitude of the mutation is given by the 'ma' argument.
        
        Args:
            ma : float in (0,1]. Max. amplitude as fraction of the parameter's range 
            name_list: list with names of candidate parameters.
    """
    n = np.random.randint(len(name_list))
    par_name = name_list[n]
    l = ranges[par_name]['low']
    h = ranges[par_name]['high']
    cfg[par_name] = max(l, min(h, cfg[par_name] + ma * (h-l) * (np.random.random()-0.5)))

In [6]:
# Create the initial population: method 1

# create variations of the parameters we want to investigtate
pop_size = 90 # number of configurations in the population
pop = [{} for _ in range(pop_size)]
# first fill the default values in n_def elements
n_def = 3
for ind in range(n_def):
    for name in par_list:
        pop[ind][name] = ranges[name]['default']
        
# fill the rest with variations in both directions
chg_name = ["low", "default", "high"] # auxiliary list
for ind in range(n_def, pop_size):
    chg_dirs = [chg_name[i] for i in np.random.randint(3, size=len(par_list))]
    for idx, name in enumerate(par_list):
        pop[ind][name] = 0.5 * (ranges[name][chg_dirs[idx]] + ranges[name]["default"])
        
# Throw n_muts mutations, and n_soft_muts soft mutations
n_muts = 20
n_soft_muts = 10 # must have n_muts + n_soft_muts < pop_size-1
perm = np.random.permutation(range(1,pop_size)) # don't mutate the first element
for i in range(n_muts):
    mutate(pop[perm[i]])
for i in range(n_muts,n_muts+n_soft_muts):
    soft_mutate(pop[perm[i]], 0.2)

In [7]:
# Create the initial population: method 2

# Load the results from a previous run
fname = '/home/z/projects/draculab/saves/v3ft3p2ph2_pop_2021-05-28__10_53'
with (open(fname, "rb")) as f:
    prev_pop = pickle.load(f)
    f.close()

# If the results are from a run with fewer parameters
# fill it with the default values.
for cfg in prev_pop:
    for name in ranges:
        if not name in cfg:
            cfg[name] = ranges[name]['default']

In [8]:
# Mix configurations from the two methods
pop[50:] = prev_pop[:40]

# If configurations have fewer parameters
# fill them with the default values.
for cfg in pop:
    for name in ranges:
        if not name in cfg:
            cfg[name] = ranges[name]['default']

In [9]:
# reset fitness and number of evaluations
for dic in pop:
    dic['fitness'] = None # average fitness value
    dic['n_evals'] = 0  # number of times fitness has been evaluated
    
# Set search parameters present in the configurations
for cfg in pop:
    cfg['t_pres'] = 40
    cfg['par_heter'] = 0.01

In [10]:
# print used configuration
for dic in pop[0:3]:
    print('{',end='')
    for name in dic.keys():
        print("\'%s\':%s, "%(name, dic[name]), end='')
#         if name in net_conf:
#             print("\'%s\':%s, "%(name, dic[name]), end='')
#         if name != 'fitness' or dic['fitness'] != None:
#             print("\'%s\':%.2f, " % (name, dic[name]), end='')
    print('}\n')

pop[0]

{'A__M_lrate':6.0, 'A__M_w_max_frac':0.33, 'A__M_w_sum':2.0, 'C_adapt_amp':5.0, 'C_cid':0.15, 'C_sigma':0.3, 'M__C_lrate':300.0, 'M__C_w_sum':3.0, 'M_cid':0.3, 'M_des_out_w_abs_sum':2.0, 'g_e_factor':2.0, 'C_slope':2.5, 'C_thresh':0.7, 'AL_thresh':0.4, 'b_e':2.0, 'C__C_antag':1.5, 'C__C_p_antag':0.25, 'C__C_p_syne':0.3, 'C__C_syne':0.5, 'CE__CI_w':0.5, 'CI__CE_w':-1.5, 'M__M_w':-0.5, 'SPF__SPF_w':-0.5, 'fitness':None, 'n_evals':0, 't_pres':40, 'par_heter':0.01, }

{'A__M_lrate':6.0, 'A__M_w_max_frac':0.33, 'A__M_w_sum':2.0, 'C_adapt_amp':5.0, 'C_cid':0.15, 'C_sigma':0.3, 'M__C_lrate':300.0, 'M__C_w_sum':3.0, 'M_cid':0.3, 'M_des_out_w_abs_sum':2.0, 'g_e_factor':2.0, 'C_slope':2.245900063549469, 'C_thresh':0.7, 'AL_thresh':0.4, 'b_e':2.0, 'C__C_antag':1.5, 'C__C_p_antag':0.25, 'C__C_p_syne':0.3, 'C__C_syne':0.5, 'CE__CI_w':0.5, 'CI__CE_w':-1.5, 'M__M_w':-0.5, 'SPF__SPF_w':-0.5, 'fitness':None, 'n_evals':0, 't_pres':40, 'par_heter':0.01, }

{'A__M_lrate':6.0, 'A__M_w_max_frac':0.33, 'A__M

{'A__M_lrate': 6.0,
 'A__M_w_max_frac': 0.33,
 'A__M_w_sum': 2.0,
 'C_adapt_amp': 5.0,
 'C_cid': 0.15,
 'C_sigma': 0.3,
 'M__C_lrate': 300.0,
 'M__C_w_sum': 3.0,
 'M_cid': 0.3,
 'M_des_out_w_abs_sum': 2.0,
 'g_e_factor': 2.0,
 'C_slope': 2.5,
 'C_thresh': 0.7,
 'AL_thresh': 0.4,
 'b_e': 2.0,
 'C__C_antag': 1.5,
 'C__C_p_antag': 0.25,
 'C__C_p_syne': 0.3,
 'C__C_syne': 0.5,
 'CE__CI_w': 0.5,
 'CI__CE_w': -1.5,
 'M__M_w': -0.5,
 'SPF__SPF_w': -0.5,
 'fitness': None,
 'n_evals': 0,
 't_pres': 40,
 'par_heter': 0.01}

In [11]:
# A function that evaluates the fitness of a given configuration
def eval_config(cfg):
    """ Returns the error for a network with a given configuration.

        Args:
            cfg : a configuration dictionary.
        Returns:
            error : A float calculated from the sum of activities in the SPF layer.
    """
    np.random.seed() # will try to get a seed from /dev/urandom
    if cfg['n_evals'] > max_evals: # if the fitness has been evaluated "enough" times. See cell below...
        return cfg['fitness']
    t_pres = cfg['t_pres']
    
    # obtain a network with the given configuration
    net, pops_dict, hand_coords, m_idxs = net_from_cfg(cfg,
               t_pres = cfg['t_pres'], 
               par_heter = cfg['par_heter'],
               set_C_delay = False,
               rand_targets = True,
               C_noise = True,
               track_weights = False,
               track_ips = False)
    # run the network
    run_time = 700.
    #start_time = time.time()
    times, data, plant_data  = net.flat_run(run_time)
    #print('Execution time is %s seconds' % (time.time() - start_time))

    # calculate average error in last half of reaching
    P = pops_dict['P']
    arm_activs = plant_data[P]
    plant = net.plants[P]
    # modified copy-paste of plt.upd_ip_impl
    q1 = arm_activs[:,0]
    q2 = arm_activs[:,2]
    q12 = q1+q2
    c_elbow = np.array((plant.l_arm*np.cos(q1), plant.l_arm*np.sin(q1)))
    c_hand = np.array((c_elbow[0] + plant.l_farm*np.cos(q12),
                    c_elbow[1] + plant.l_farm*np.sin(q12))).transpose()
    coord_idxs = np.floor(times/t_pres).astype(int)
    des_coords = np.array(hand_coords)[m_idxs[coord_idxs],:] # desired coordinates at each moment in time

    error_time = run_time - round(run_time/2.)
    error_idx = int(round(error_time/net.min_delay))
    hand_error = np.linalg.norm(c_hand-des_coords, axis=1)
    hand_error_integ = hand_error[error_idx:].sum()
    avg_hand_error = hand_error_integ / (hand_error.size - error_idx)

    #return hand_error_integ
    return avg_hand_error

---
# Genetic algorithm  

---

In [13]:
# A function to produce offspring by crossing individuals
par_names = list(pop[0].keys()) # list with all parameter names

def create_offspring(cfg1, cfg2, par_list=par_names):
    """ Given 2 configurations, return 2 offspring from random swapping.
    
        To produce offspring, first we choose one split point in the
        dictionary. The first offspring has the values of cfg1 up to that
        point, and cfg2 afterwards. The second offspring has the cfg2 values
        up to the split point, and cfg1 afterwards. Since the dictionaries are
        not ordered, we use a parameter list to set the split point.
    
        Args:
            cfg1, cfg2: parameter dictionaries
            par_list: list with the keys in cfg1, cfg2
        Returns:
            cfg3, cfg4: dictionaries from swapping values in cfg1, cfg2
    """
    if focus_params:
        par_list = main_pars
    sp = np.random.randint(len(par_list))# split point as an index to par_list
    cfg3 = cfg1.copy()
    cfg4 = cfg2.copy()
    for i in range(sp, len(par_list)):
        cfg3[par_list[i]] = cfg2[par_list[i]]
        cfg4[par_list[i]] = cfg1[par_list[i]]
    return cfg3, cfg4

# visualize
# cfg3, cfg4 = create_offspring(pop[10], pop[11])
# for dic in (pop[10], pop[11], cfg3, cfg4):
#     print('{',end='')
#     for name in par_names:
#         if name != "fitness":
#             print("\'%s\':%.2f, " % (name, dic[name]), end='')
#     print('}\n')


In [14]:
####################################
###### The genetic algorithm ######
####################################

#pop = pop[0:15] # limit pop size for debugging
n_mates = 30 # number of individuals to mate at each generation (even number)
max_gens = 80 # maximum number of generations
n_soft_mut = 10 # numbef of individuals to soft-mutate per generation
r_soft_mut = 0.2 # relative amplitude of soft mutations
n_mut = 8 # number of individuals to mutate per generation
n_procs = 28 # number of processes to use for fitness evaluation
n_save = 3 # number of individuals to protect from replacement and mutation
use_pso = True # whether to insert configurations from the PSO algorithm

# setting name for file where parameters will be stored
path = "/home/z/Dropbox (OIST)/saves/"
#path = "/home/z/projects/draculab/saves/"
fname1 = "gene"
fname2 = datetime.now().strftime('%Y-%m-%d')
fname = path + fname1 + "_" + fname2

pop_len = len(pop)

for gen in range(max_gens):
    start_time = time.time()
    # 1) Evaluate fitness
    # 1.1) Do the evaluation
    ######### Single process version
    #fits = list(map(eval_config, pop))
    ######## parallel version
    with Pool(n_procs) as p:
        fits = list(p.map(eval_config, pop))
        p.close()
        p.join()
    #print(fits)
    # 1.2) update the average fitness values
    for idx, cfg in enumerate(pop):
        nr = cfg['n_evals'] # n_evals is not updated yet...
        #print(nr)
        #print(fits[idx])
        #print(cfg['fitness'])
        if nr > 0:
            if nr <= max_evals:
                cfg['fitness'] = (cfg['fitness']*nr + fits[idx])/(nr+1)
        else:
            cfg['fitness'] = fits[idx]
        cfg['n_evals'] = cfg['n_evals'] + 1
    #---------------------------------------------------------------------------------
    # 1.3) Inserting configurations from the PSO algorithm
    if use_pso:
        pso_name = path + "pso_" + fname2
        try:
            with open(pso_name, 'rb') as f:
                pso_pop = pickle.load(f)
                f.close()
            ext_pop = pop + pso_pop
            print("Mixed pso and gene pops!!!")
        except IOError:
            if gen > 2: # if the pso algorithm should likely be done
                from warnings import warn
                warn('population ' + pso_name + 'could not be imported',
                     UserWarning)
            ext_pop = pop
    #---------------------------------------------------------------------------------
    # 2) Sort according to fitness. Lowest error first.
    if use_pso:
        pop = sorted(ext_pop, key=lambda d: d['fitness'])[:pop_len]
    else:
        pop = sorted(pop, key=lambda d: d['fitness'])
    # 2.1) Save current generation
    with open(fname, 'wb') as f:
        pickle.dump(pop, f)
        f.close()
    # 2.2) A quick message
    print("Generation %d evaluated. Best fitness: %.3f"%(gen,pop[0]['fitness']))
    print("Mean fitness = %.3f"%(np.mean(np.array(fits))))
    # 2.3) If best fitness good enough, break
    if pop[0]['fitness'] < target_fitness:
        print("Good enough parameters found. Stopping search.")
        break
    # 3) mate and replace
    # 3.1) Select individuals to be replaced with probability proportional to error
    fits.sort() # sort the fitnesses (now in the same order as pop)
    fits = np.array(fits)
    fits = fits/fits.sum() # normalize fitnesses so they add to 1
    cumsum_fits = fits[:] # cumsum_fits[i] = sum(fits[:i])
    for i in range(1,len(cumsum_fits)):
        cumsum_fits[i] = cumsum_fits[i-1] + cumsum_fits[i]
    repl_list = [] # list with indexes of individuals to be replaced
    while len(repl_list) < n_mates:
        min_r = cumsum_fits[n_save] # don't replace the first n_save individuals
        r = min_r + (1.-min_r) * np.random.random()
        candidate = n_save
        for i in range(n_save, len(fits)):
            if cumsum_fits[i] > r:
                break
            candidate += 1
        if candidate in repl_list:
            continue
        else:
            repl_list.append(candidate)
    print("to replace: ", end='')
    print(repl_list)
    # 3.2) Arrange individuals in random pairs
    perm = np.random.permutation(n_mates) # this will do 
    # 3.3) mate
    new_pops = []
    for i in range(int(np.floor(n_mates/2))):
        off1, off2 = create_offspring(pop[perm[2*i]], pop[perm[2*i+1]])
        new_pops.append(off1)
        new_pops.append(off2)
    # 3.4) replace
    for i, cfg in enumerate(new_pops):
        pop[repl_list[i]] = cfg
    # 4) mutate
    # 4.1) soft mutations
    for _ in range(n_soft_mut):
        idx = np.random.randint(len(pop))
        if idx < n_save:
            copy = pop[idx].copy()
            soft_mutate(copy, r_soft_mut)
            pop[-idx-1] = copy
            pop[-idx-1]['fitness'] = None
            pop[-idx-1]['n_evals'] = 0
        else:
            soft_mutate(pop[idx], r_soft_mut)
            pop[idx]['fitness'] = None
            pop[idx]['n_evals'] = 0
    # 4.2) mutations
    # 4.2.1) select individuals to mutate
    # sq_fits = fits*fits
    #cumsum_sq_fits = fits * fits # cumsum_sq_fits[i] = sum(sq_fits[:i])
    cumsum_sq_fits = fits # proportional to fits, rather than its square
    cumsum_sq_fits = cumsum_sq_fits / cumsum_sq_fits.sum()
    for i in range(1,len(cumsum_sq_fits)):
        cumsum_sq_fits[i] = cumsum_sq_fits[i-1] + cumsum_sq_fits[i]
    mut_list = [] # list with indexes of individuals to be mutate
    while len(mut_list) < n_mut:
        r = np.random.random()
        candidate = 0
        for i in range(len(fits)):
            if cumsum_sq_fits[i] > r:
                break
            candidate += 1
        if candidate in mut_list:
            continue
        else:
            mut_list.append(candidate)
    print("to mutate: ", end='')
    print(mut_list)
    for idx in mut_list:
        if idx < n_save:
            copy = pop[idx].copy()
            mutate(copy)
            pop[-idx-1] = copy
            pop[-idx-1]['fitness'] = None
            pop[-idx-1]['n_evals'] = 0
        else:
            mutate(pop[idx])
            pop[idx]['fitness'] = None
            pop[idx]['n_evals'] = 0
            
    print('generation %d finished in %s seconds' % (gen, time.time() - start_time))

    

Generation 0 evaluated. Best fitness: 0.068
Mean fitness = 0.241
to replace: [69, 85, 65, 74, 61, 62, 67, 80, 56, 4, 70, 31, 50, 84, 34, 59, 46, 29, 81, 73, 60, 5, 38, 75, 51, 66, 58, 83, 87, 78]
to mutate: [81, 42, 70, 71, 79, 46, 47, 87]
generation 0 finished in 6583.3664219379425 seconds
Mixed pso and gene pops!!!
Generation 1 evaluated. Best fitness: 0.073
Mean fitness = 0.173
to replace: [65, 23, 77, 11, 75, 19, 52, 88, 27, 48, 22, 71, 57, 45, 83, 44, 73, 80, 56, 62, 55, 76, 33, 67, 89, 31, 68, 58, 12, 14]
to mutate: [69, 55, 76, 60, 80, 86, 71, 73]
generation 1 finished in 6744.355035543442 seconds


KeyError: 'b_e'

In [23]:
for cfg in pop:
    if not 'b_e' in cfg:
        print(len(cfg))
        
print(len(pop[0]))

17
17
17
27


---
# Particle Swarm Optimization
---

In [9]:
# Utility functions for the PSO algorithm
# These methods assume that 'par_list' and 'ranges' have been defined.
ref_cfg = pop[0] # a configuration with all values, including those that
                 # are not in par_list
    
def add_to_cfg(cfg, vel):
    """ Add a vector to the values of a configuration.
    
        Args:
            cfg : a configuration dictionary
            vel : Numpy array of values to be added
            
        The values in 'vel' are in the order of 'par_list'.
        Each entry in vel is a value in the [-mv, mv] interval, where mv
        is a parameter indicating the maximum velocity.
        
        For each parameter 'par_name' in 'par_list', the corresponding
        value in vel will be multiplied times
        (ranges[par_name]['high'] - ranges[par_name]['low'])
        before being added.
        
        If a value exceeds the limits set in 'ranges' it will be clipped.
    """
    for idx, name in enumerate(par_list):
        cfg[name] += vel[idx] * (
                       ranges[name]['high'] - ranges[name]['low'])
        cfg[name] = max( min(ranges[name]['high'], cfg[name]),
                           ranges[name]['low'])
        
def add_vel(vel, acc, mv=0.5):
    """ Add an acceleration to a velocity vector.
    
        Args:
            vel : velocity vector (numpy array).
            acc : acceleration vector (numpy array).
            mv : maximum velocity.
        Returns:
            vel + acc
            
        The entries of 'vel' and 'acc' are in the order of 'par_list'.
        If vel[i] + acc[i] > mv, or vel[i] + acc[i] < -mv, values will be
        clipped.
    """
    return np.maximum(np.minimum(vel + acc, mv), -mv)
    
def cfg_to_vec(cfg):
    """ Convert a configuration dictionary to a vector.
    
        Args:
            cfg: configuration dictionary.
        Returns:
            vec: a numpy array with the values of the configuration.
            
        The order of the values in 'vec' is that of 'par_list'.
        The 'cfg' entries for 'fitness', 'n_evals', 't_pres', and
        'par_heter' will be omitted in 'vec'.
    """
    vec = np.zeros(len(par_list))
    for idx, name in enumerate(par_list):
        vec[idx] = cfg[name]
    return vec

def vec_to_cfg(vec, fitness=None, n_evals=0, t_pres=None, par_heter=0.01):
    """ Convert a vector to a configuration dictionary.
    
        Args:
            vec: array-like with the values for the configuration.
            'fitness', 'n_evals', 't_pres', 'par_heter': values not present
                in 'vec' that will be appendend to the configuration dict.
        Returns:
            cfg: a configuration dictionary with the values in vec.
            
        The order of the values in vec should be that of 'par_list'.
    """
    cfg = ref_cfg
    for val, name in zip(vec, par_list):
        cfg[name] = val
    cfg['fitness'] = fitness
    cfg['n_evals'] = n_evals
    cfg['t_pres'] = t_pres
    cfg['par_heter'] = par_heter
    return cfg


In [11]:
###############################
###### The PSO algorithm ######
###############################

W = 0.4 # inertia weight
c1 = 0.6 # weight of accel towards personal best
c2 = 0.2 # weight of accel towards global best
mv = 0.4 # maximum velocity (relative to range witdth of paramter)

use_gene = False # insert particles from a concomitant genetic algorithm
sig = lambda f: 1./(1. + np.exp(4.*(f + 0.1))) # to set probability of insertion

max_iters = 80 # maximum number of iterations
n_procs = 15 # number of processes to use for fitness evaluation

g_best_f = 1e10 # best fitness so far
p_best_fs = [1e10] * len(pop) # best personal fitnesses
t_pres = pop[0]['t_pres'] # assuming all presentation times are equal
vels = np.random.random((len(pop), len(par_list))) - 0.5 # initial velocities

g_best = cfg_to_vec(pop[0]) # arbitrary initialization of global best
p_bests = np.zeros((len(pop), len(par_list)))
for idx, cfg in enumerate(pop):
    p_bests[idx,:] = cfg_to_vec(cfg)

# setting name for file where parameters will be stored
#path = "/home/z/projects/draculab/saves/"
path = "/home/z/Dropbox (OIST)/saves/"
fname1 = "pso"
fname2 = datetime.now().strftime('%Y-%m-%d')
fname = path + fname1 + "_" + fname2

for itr in range(max_iters):
    start_time = time.time()
    
    # 1) Evaluate fitness
    # 1.1) Do the evaluation
    ######### Single process version
    #fits = list(map(eval_config, pop))
    ######## parallel version
    with Pool(n_procs) as p:
        fits = list(p.map(eval_config, pop))
        p.close()
        p.join()
    #print(fits)
    # 1.2) update the average fitness values
    # In PSO configurations change every iteration, so n_evals is 
    # either 0 or 1. However, some configuration may have come from the
    # genetic algorithm, so we consider the case n_evals > 0
    for idx, cfg in enumerate(pop):
        nr = cfg['n_evals'] # n_evals is not updated yet...
        if nr > 0:
            if nr <= max_evals: 
                cfg['fitness'] = (cfg['fitness']*nr + fits[idx])/(nr+1)
        else:
            cfg['fitness'] = fits[idx]
        cfg['n_evals'] = cfg['n_evals'] + 1
        
    # 2) Find indexes that sort according to fitness. Lowest error first.
    srt_idx = np.argsort(fits)
    
    # 3.1) Update g_best
    if fits[srt_idx[0]] < g_best_f:
        g_best_f = fits[srt_idx[0]]
        g_best = cfg_to_vec(pop[srt_idx[0]])
        
    for idx, cfg in enumerate(pop):
        cfg_vec = cfg_to_vec(cfg)
        # 3.2) Update p_bests
        if cfg['fitness'] < p_best_fs[idx]:
            p_best_fs[idx] = cfg['fitness']
            p_bests[idx] = cfg_vec
            
        # 4) Update velocities
        r1, r2 = np.random.random(2)
        vels[idx] = add_vel(W * vels[idx], c1 * r1 * (p_bests[idx] - cfg_vec))
        vels[idx] = add_vel(vels[idx], c2 * r2 * (g_best - cfg_vec))
    
    # 5) Save current iteration
    best_pop = [vec_to_cfg(g_best, fitness=g_best_f, n_evals=1, t_pres=t_pres)] + pop
    with open(fname, 'wb') as f:
        pickle.dump(best_pop, f)
        f.close()
    
    # A quick message
    print("Iteration %d evaluated. Best fitness: %.3f"%(itr, g_best_f))
    print("Mean fitness = %.3f"%(np.mean(np.array(fits))))
    # 2.3) If best fitness good enough, break
    if g_best_f < target_fitness:
        print("Good enough parameters found. Stopping search.")
        break
    
    # 6) Update particles
    for idx, cfg in enumerate(pop):
        add_to_cfg(cfg, vels[idx])
        
    #---------------------------------------------------------------------------------
    # 7) Inserting a configuration from the genetic algorithm
    if use_gene:
        g_name = path + "gene_" + fname2
        try:
            with open(g_name, 'rb') as f:
                gene_pop = pickle.load(f)
                f.close()
            gene_cfg = gene_pop[0]
            in_prob = sig(g_best_f - gene_cfg['fitness']) # insertion probability
            if np.random.random() < in_prob:
                rem_idx = np.random.randint(len(pop))
                pop[rem_idx] = gene_cfg
                fits[rem_idx] = gene_cfg['fitness']
                print("inserted pop " + str(rem_idx) + "!!!")    
        except IOError:
            if itr > 2: # if the genetic algorithm should likely be done
                from warnings import warn
                warn('population ' + g_name + 'could not be imported',
                     UserWarning)
                
    print('Iteration %d finished in %s seconds' % (itr, time.time() - start_time))
            

Iteration 0 evaluated. Best fitness: 0.116
Mean fitness = 0.222
Iteration 0 finished in 27.140151500701904 seconds
Iteration 1 evaluated. Best fitness: 0.116
Mean fitness = 0.229
Iteration 1 finished in 27.320566415786743 seconds
Iteration 2 evaluated. Best fitness: 0.043
Mean fitness = 0.198
Iteration 2 finished in 26.635707139968872 seconds


In [12]:
# print final population
for dic in pop[:5]:
    print('{',end='')
    for name in dic.keys():
        if name != 'fitness' or dic['fitness'] != None:
            print("\'%s\':%.2f, " % (name, dic[name]), end='')
    print('}\n')

pop[0]

{'A__M_lrate':2.73, 'A__M_w_max_frac':0.21, 'A__M_w_sum':21.50, 'AL_thresh':-0.08, 'b_e':3.88, 'C__C_antag':1.42, 'C__C_p_antag':0.74, 'C__C_p_syne':0.35, 'C__C_syne':0.33, 'C_adapt_amp':11.41, 'C_cid':0.10, 'C_sigma':0.48, 'C_slope':1.77, 'C_thresh':0.35, 'CE__CI_w':0.71, 'CI__CE_w':-1.83, 'g_e_factor':1.71, 'M__C_lrate':343.75, 'M__C_w_sum':5.47, 'M__M_w':0.00, 'M_cid':0.93, 'M_des_out_w_abs_sum':1.46, 'SPF__SPF_w':-0.07, 'fitness':0.34, 'n_evals':1.00, 't_pres':40.00, 'par_heter':0.01, }

{'A__M_lrate':5.76, 'A__M_w_max_frac':0.50, 'A__M_w_sum':21.50, 'AL_thresh':0.25, 'b_e':1.44, 'C__C_antag':1.51, 'C__C_p_antag':0.54, 'C__C_p_syne':0.38, 'C__C_syne':0.84, 'C_adapt_amp':15.00, 'C_cid':0.12, 'C_sigma':0.12, 'C_slope':2.84, 'C_thresh':0.80, 'CE__CI_w':1.10, 'CI__CE_w':-1.93, 'g_e_factor':1.96, 'M__C_lrate':337.44, 'M__C_w_sum':5.45, 'M__M_w':0.00, 'M_cid':0.99, 'M_des_out_w_abs_sum':2.15, 'SPF__SPF_w':-0.40, 'fitness':0.13, 'n_evals':1.00, 't_pres':40.00, 'par_heter':0.01, }

{'A__M_

{'A__M_lrate': 2.7341357230234973,
 'A__M_w_max_frac': 0.20605804471513278,
 'A__M_w_sum': 21.5,
 'AL_thresh': -0.08179848000533585,
 'b_e': 3.8828035128252716,
 'C__C_antag': 1.4228292992516052,
 'C__C_p_antag': 0.7397524146335008,
 'C__C_p_syne': 0.3516800011288286,
 'C__C_syne': 0.325460593028586,
 'C_adapt_amp': 11.410677400994746,
 'C_cid': 0.1,
 'C_sigma': 0.48150726198995586,
 'C_slope': 1.7696225876919747,
 'C_thresh': 0.3455782249196093,
 'CE__CI_w': 0.7102217806543719,
 'CI__CE_w': -1.828816851437376,
 'g_e_factor': 1.7114423657602078,
 'M__C_lrate': 343.7463363436077,
 'M__C_w_sum': 5.470939880845892,
 'M__M_w': 0.0,
 'M_cid': 0.9325076280660229,
 'M_des_out_w_abs_sum': 1.4606924993186237,
 'SPF__SPF_w': -0.06690836645601457,
 'fitness': 0.3389436976213204,
 'n_evals': 1,
 't_pres': 40,
 'par_heter': 0.01}

In [ ]:
# Save the final population
fname = "v3_nst_afx_pop"
fname += "_" + datetime.now().strftime('%Y-%m-%d__%H_%M')
with open(fname, 'wb') as f:
    pickle.dump(pop, f)
    f.close()

In [ ]:
# Load a saved population
import pickle
fname = 'v3_nst_afx_pop'
with (open(fname, "rb")) as f:
    results = pickle.load(f)
    f.close()

In [ ]:
# Test a configuation
cfg_id = 0 # index in the population for the configuration
net, pops_dict, hand_coords, m_idxs, t_pres, _  = net_from_cfg(pop[cfg_id])
pops_names = ['SF', 'SP', 'SPF', 'AL', 'AF', 'SP_CHG', 'CE', 'CI', 'M', 'ACT', 'P']
for name in pops_names:
    exec("%s = %s"% (name, str(pops_dict[name])))

start_time = time.time()
times, data, plant_data  = net.flat_run(600.)
#times, data, plant_data  = net.run(40.)
print('Execution time is %s seconds' % (time.time() - start_time))
data = np.array(data)
#Execution time is 8.687349319458008 seconds  << before sc_inp_sum_mp, flat_run(5.)

Same history as `v3_nst_afx`
...
